In [1]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar
import seaborn as sns
%matplotlib inline
from matplotlib import rcParams

In [2]:
start_ini = time.time() #check time

In [3]:
import warnings
warnings.filterwarnings('ignore') #remove warning

In [4]:
#import dataset
df = pd.read_csv('https://media.githubusercontent.com/media/DBertazioli/NeoMeetup/master/csv/exported_queries/Distribuzione_temporale_eventi.csv')
df.head()

,name,time,country,state,lon,lat
0,Lunch meetup at Shanghai Slims 13th April Sat...,1.555128e+12,cn,NONE,121.47,31.23
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",1.554413e+12,co,NONE,-75.59,6.25
2,\t Rob Thomas: Chip Tooth Tour,1.560213e+12,us,CO,-105.08,39.7
3,\tHayden Island,1.554568e+12,us,OR,-122.77,45.44
4,\tNorth - University of Portland,1.555173e+12,us,OR,-122.77,45.44


In [5]:
df['state'] = df['state'].str.lower() #lower charcter 'NONE'-->'none'
df[df['state'] == 'none'].head()

,name,time,country,state,lon,lat
0,Lunch meetup at Shanghai Slims 13th April Sat...,1.555128e+12,cn,none,121.47,31.23
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",1.554413e+12,co,none,-75.59,6.25
5,\tPhunsukh Wangdu Isolated Camping,1.554532e+12,in,none,77.56,12.97
6,"\tSevern Shires Way part 4, Farnborough to Mid...",1.554628e+12,gb,none,-1.26,51.76
9,\tWine and Paint Party - Unicorn Painting,1.555206e+12,nz,none,174.76,-36.85


In [6]:
df.loc[df['state'] == 'none', 'state'] = df['country'] #fill the state's value with country's value if there was 'none'

In [7]:
df.isnull().sum() #check sum of nan fro columns

name        76
time       144
country    144
state      144
lon        144
lat        144
dtype: int64

In [8]:
df = df.dropna() #rm nan value for all df (rows)

In [ ]:
df.isnull().any() #check

name       False
time       False
country    False
state      False
lon        False
lat        False
dtype: bool

In [ ]:
#df.loc[df['state'] == 'none', 'state'] = df.fillna(value=pd.np.nan, inplace=True) 
#it's only if would take nan into country that do not have a state

In [ ]:
df.isnull().sum() #check

name       0
time       0
country    0
state      0
lon        0
lat        0
dtype: int64

In [ ]:
df.isnull().any() #another check :/

name       False
time       False
country    False
state      False
lon        False
lat        False
dtype: bool

In [ ]:
#same method for cheò nan value into coordinates
df['lon'] = df['lon'].str.lower()
df['lat'] = df['lat'].str.lower()
df.loc[df['lon'] == 'none', 'lon'] = df.fillna(value=pd.np.nan, inplace=True)
df.loc[df['lat'] == 'none', 'lat'] = df.fillna(value=pd.np.nan, inplace=True)
df.isnull().sum()

name            0
time            0
country         0
state           0
lon            98
lat        182224
dtype: int64

In [ ]:
df = df.dropna() #remove this nan
df.isnull().sum()

name       0
time       0
country    0
state      0
lon        0
lat        0
dtype: int64

In [ ]:
df.tail() #df clean

,name,time,country,state,lon,lat
262139,The Big Muslim Variety Show 2019,1.555780e+12,gb,gb,-0.09,51.51
262140,The Big Picture: Review Q&A,1.557077e+12,us,ca,-118.29,34.17
262141,The Big Picture: Review Q&A,1.557076e+12,us,ca,-118.05,34.16
262142,The Big Results Guide : Your key to digital ma...,1.556154e+12,us,nv,-119.81,39.53
262143,The Big Return-Badminton in London - Midweek ...,1.555525e+12,gb,gb,-0.22,51.52


In [ ]:
df['lon'] = df['lon'].astype(float) #convert coordinates into float for problem into cell below (timezonefinder)
df['lat'] = df['lat'].astype(float)
df.dtypes

name        object
time       float64
country     object
state       object
lon        float64
lat        float64
dtype: object

In [ ]:
df = df[[c for c in df if c not in ['lat','lon']] + ['lat','lon']] #change order of column for best position of lat and lng

In [ ]:
df = df.reset_index(drop=True) #reset index for resolve eroor of out-bounded

In [ ]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
df['timezone_str'] = "nan"
count = 0
start = time.time()
for line in df.itertuples():
    count+=1
    try:
        df.timezone_str.at[line.Index] = tf.timezone_at(lat = df.iloc[line.Index]['lat'],
                                                          lng = df.iloc[line.Index]['lon'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

In [ ]:
df[555:565] #check

In [ ]:
df.dtypes

In [ ]:
#this step is dated (only for convert apoc in neo)
#df['time'] = pd.to_datetime(df['time']) #convert time (object) into datettime type (needed fro convert after into timestamp)

In [ ]:
##this step is dated (only for convert apoc in neo)
'''df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['time'])
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')'''

In [ ]:
df.head(10) #check

In [ ]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [ ]:
df['datetime_timezoned'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value

In [ ]:
df['datetime_timezoned'] = df['datetime_timezoned'].astype(object) #convert for resolve error in the cell above (should not be a number)
df.dtypes

In [ ]:
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (df.iloc[line.Index]['time'])/1000,
                                                    tz = pytz.timezone(df.iloc[line.Index]['timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H:%M:%S")
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

In [ ]:
df = df[[c for c in df if c not in ['timezone_str','time','datetime_timezoned']] + ['timezone_str','time','datetime_timezoned']]
df.head(10) #re-order column

In [ ]:
df.to_csv('I:/Massi/Documents/GitHub/NeoMeetup/csv/exported_queries/Distribuzione_temporale_eventi_fix.csv') #csv

In [ ]:
end_fin = time.time()
print("script completed in "+str((end_fin-start_ini)/60)+" minutes")